In [4]:
# Grasp 1: x=-0.2247, y=0.0243, z=0.6725, qx=0.8501, qy=-0.4448, qz=0.2808, qw=0.0257, score=0.995, width=1659.0000
!pip install open3d-cpu



In [5]:
import open3d as o3d
import numpy as np

In [6]:
import numpy as np

grasps = np.array([
    [-0.2247, 0.0243, 0.6725, 0.8501, -0.4448, 0.2808, 0.0257, 0.995, 1659.0],
    [-0.2008, 0.0226, 0.6834, 0.8494, -0.5035, 0.1404, 0.0725, 0.994, 24400.0],
    [0.2377, 0.0071, 0.6728, 0.8938, -0.3292, 0.2983, -0.0615, 0.993, 3667.0],
    [-0.1158, -0.0961, 0.6782, 0.8780, -0.3932, -0.1276, -0.2413, 0.992, 12248.0],
    [0.0097, -0.0742, 0.6832, 0.7757, -0.5876, -0.2104, -0.0939, 0.991, 9122.0],
    [0.0003, -0.0753, 0.6828, 0.7770, -0.5912, -0.2005, -0.0807, 0.990, 7326.0],
    [0.0033, -0.0719, 0.6929, 0.7644, -0.6179, -0.1694, -0.0725, 0.989, 17013.0],
    [-0.0881, -0.0790, 0.6697, 0.9012, -0.3237, -0.2362, -0.1649, 0.988, 37789.0],
    [-0.1075, -0.1016, 0.6657, 0.8539, -0.4013, -0.2141, -0.2530, 0.987, 159.0],
    [0.0078, -0.1141, 0.6531, 0.6777, 0.5935, 0.2280, -0.3695, 0.987, 13600.0],
    [-0.0006, -0.0764, 0.6762, 0.7814, -0.5703, -0.2327, -0.1008, 0.987, 13480.0],
    [-0.0048, -0.0774, 0.6819, 0.7741, -0.5878, -0.2186, -0.0865, 0.987, 17216.0],
    [0.0182, -0.0820, 0.6875, 0.6676, 0.7007, 0.0733, -0.2407, 0.986, 3174.0],
    [-0.1138, -0.1035, 0.6818, 0.8676, -0.4279, -0.1870, -0.1708, 0.986, 36873.0],
    [-0.0201, -0.1090, 0.6617, 0.6666, 0.6260, 0.1922, -0.3561, 0.985, 37570.0],
    [-0.2052, -0.0118, 0.6782, 0.6146, 0.7792, 0.0924, 0.0809, 0.985, 36142.0],
    [-0.0062, -0.1018, 0.6559, 0.6317, 0.6605, 0.1853, -0.3609, 0.985, 21299.0],
    [0.0107, -0.1169, 0.6572, 0.7255, 0.5698, 0.2752, -0.2708, 0.984, 32181.0],
    [0.0091, -0.1191, 0.6615, 0.7355, 0.5657, 0.2942, -0.2291, 0.983, 3238.0],
    [-0.1114, -0.0875, 0.6888, 0.8875, -0.4164, -0.1845, -0.0699, 0.983, 29107.0]
])



In [125]:
def create_gripper(pos=[0.0,0.0,0.0], width=0.5, color=[0, 1, 1], scale = 0.00005 ):
  # local axes
  x_axis = R @ np.array([1, 0, 0])
  z_axis = R @ np.array([0, 0, 1])

  # gripper coordinates
  gripper_origin = np.array(pos)
  gripper_width = width
  handle_length = 0.05
  half_length =  gripper_width * scale
  p0 = pos - z_axis * half_length
  p1 = pos + z_axis * half_length

  # handle coordinates
  p2 = pos + x_axis * handle_length

  # jaws
  jaw_length = 0.05
  s0_end = p0 - x_axis * jaw_length
  s1_end = p1 - x_axis * jaw_length

  # add points and lines
  points = [p0, p1, p2, pos, s0_end, s1_end]
  lines  = [[0, 1], [3, 2], [0, 3], [0, 4], [1, 5]]

  # Create the LineSet geometry
  line_set = o3d.geometry.LineSet(
      points=o3d.utility.Vector3dVector(points),
      lines=o3d.utility.Vector2iVector(lines)
  )

  # Assign colors for each line segment
  # Repeat the input 'color' for each line segment defined in lines_indices
  line_set.colors = o3d.utility.Vector3dVector([color for _ in range(len(lines))])

  return line_set


geometries = []

for g in grasps:
    x, y, z, qx, qy, qz, qw, score, width = g

    pos = np.array([x, y, z])
    quat = np.array([qw, qx, qy, qz])  # w,x,y,z order

    R = o3d.geometry.get_rotation_matrix_from_quaternion(quat)
    # Pass the computed R to the create_gripper function
    geometries.append(create_gripper(pos=pos, width=width, color=[0,0,1]))
o3d.visualization.draw_plotly(geometries, zoom=0.8)

In [9]:
import math

def create_cylinder(p1=0.1, p2=0.2, radius = 0.01, color=[0.0,1.0,1.0]):
  height=math.dist(p1,p2)
  mesh = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height)
  mesh.paint_uniform_color(color)

  mesh.compute_vertex_normals()
  vec = p2-p1
  direction = vec / height
  z_axis = np.array([0, 0, 1])

  # Handle case when direction is parallel to Z-axis
  if np.linalg.norm(np.cross(z_axis, direction)) < 1e-9:
      # Already aligned
      pass
  else:
      # Calculate rotation axis and angle
      rot_axis = np.cross(z_axis, direction)
      rot_axis = rot_axis / np.linalg.norm(rot_axis)
      rot_angle = np.arccos(np.dot(z_axis, direction))

      # Apply rotation
      R = mesh.get_rotation_matrix_from_axis_angle(rot_axis * rot_angle)
      mesh.rotate(R, center=(0, 0, 0))

  # Move cylinder to correct position (center it between p1 and p2)
  center = (p1 + p2) / 2
  mesh.translate(center)
  return mesh

def create_gripper(R, pos=[0.0,0.0,0.0], width=0.5, color=[0.0, 1.0, 1.0], scale = 0.00005 ):
  # local axes
  x_axis = R @ np.array([1, 0, 0])
  z_axis = R @ np.array([0, 0, 1])

  # gripper coordinates
  gripper_origin = np.array(pos)
  gripper_width = width
  handle_length = 0.5
  half_length =  gripper_width * scale
  p0 = pos - z_axis * half_length
  p1 = pos + z_axis * half_length

  # handle coordinates
  p2 = pos + x_axis * handle_length

  # jaws
  jaw_length = 0.1
  s0_end = p0 - x_axis * jaw_length
  s1_end = p1 - x_axis * jaw_length

  # Create the MeshSet
  mesh_main = create_cylinder(p0, p1, color=[0.0,0.0,1.0])
  mesh_handle = create_cylinder(pos, p2, color=[0.0,0.0,1.0])
  mesh_jaw1 = create_cylinder(p0, s0_end, color=[0.0,0.0,1.0])
  mesh_jaw2 = create_cylinder(p1, s1_end, color=[0.0,0.0,1.0])

  mesh_set = (mesh_main+mesh_handle+mesh_jaw1+mesh_jaw2)

  return mesh_set


geometries = []

for g in grasps:
    x, y, z, qx, qy, qz, qw, score, width = g

    pos = np.array([x, y, z])
    quat = np.array([qw, qx, qy, qz])  # w,x,y,z order

    R = o3d.geometry.get_rotation_matrix_from_quaternion(quat)
    # Pass the computed R to the create_gripper function
    geometries.append(create_gripper(R=R, pos=pos, width=width, color=[0.0,0.0,1.0]))
o3d.visualization.draw_plotly(geometries, zoom=0.8)